In [59]:
import io
import requests
from datetime import datetime

import numpy as np
import pandas as pd
import webvtt
import pyktok as pyk

pyk.specify_browser("chrome")

In [60]:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
headers = {"User-Agent": user_agent}

In [61]:
url = "https://www.tiktok.com/@tiktok/video/7361448925972155679?lang=en"

In [62]:
tt_json = pyk.alt_get_tiktok_json(url)

video_details = tt_json["__DEFAULT_SCOPE__"]["webapp.video-detail"]["itemInfo"]["itemStruct"]

for info in video_details["video"]["subtitleInfos"]:
    if info["LanguageCodeName"] == "eng-US":
        vtt_link = info["Url"]
        break

In [63]:
result = requests.get(vtt_link, headers=headers)
vtt = result.content.decode()

In [64]:
transcript = ""

for caption in webvtt.read_buffer(io.StringIO(vtt)):
    # Eng requires extra space between caption texts
    transcript += f"{caption.text} "

In [65]:
if suggested_words := video_details.get("suggestedWords"):
    suggested_words = " / ".join(suggested_words)
else:
    suggested_words = np.nan

In [66]:
meta_data = pyk.generate_data_row(video_obj=video_details)

meta_data["suggested_words"] = suggested_words
meta_data["transcript"] = transcript

meta_data.to_csv("meta.csv")